Install and load packages

In [ ]:
install.packages('readxl')
install.packages('dplyr')
install.packages('ggplot2')
install.packages('stargazer')
install.packages('purrr')
install.packages('stringr')
install.packages('lessR')
install.packages('fixest')
install.packages('ordinal')


In [2]:
library(readxl)
library(dplyr)
library(ggplot2)
library(stats)
library(stargazer)
library(purrr)
library(stringr)
library(tidyverse)
library(lessR)
library(fixest)


Warning message:
"le package 'dplyr' a 'et'e compil'e avec la version R 4.1.2"

Attachement du package : 'dplyr'


Les objets suivants sont masqu'es depuis 'package:stats':

    filter, lag


Les objets suivants sont masqu'es depuis 'package:base':

    intersect, setdiff, setequal, union


Warning message:
"le package 'ggplot2' a 'et'e compil'e avec la version R 4.1.2"
Warning message:
"le package 'stargazer' a 'et'e compil'e avec la version R 4.1.2"

Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 


Warning message:
"le package 'purrr' a 'et'e compil'e avec la version R 4.1.2"
Warning message:
"le package 'stringr' a 'et'e compil'e avec la version R 4.1.2"
-- Attaching packages --------------------------------------- tidyverse 1.3.1 --

v tibble  3.2.1     v readr   2.0.2
v tidyr   1.2.1     v forcats 0.5.1

Warning message:
"le package 'tibble' a 'et

Load df

In [ ]:
# create a list of file names and corresponding is_editorial values
file_names <- c("df_afro_final_2.xlsx", "df_alternatif_final_2.xlsx", "df_ambiance_final_2.xlsx", "df_ambient_final_2.xlsx",
                "df_amour_final_2.xlsx", "df_arabe_final_2.xlsx", "df_bienetre_final_2.xlsx", "df_blues_final_2.xlsx",
                "df_classique_final_2.xlsx", "df_country_final_2.xlsx", "df_decennies_final_2.xlsx", "df_detente_final_2.xlsx"
                )

is_editorial_values <- c(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
                         )

type_values <- c('genre', 'genre', 'mood', 'genre', 'mood', 'genre', 'mood', 'genre', 'genre', 'genre', 'trend', 'mood'
                 )

df_list <- list()
for (i in seq_along(file_names)) {
  file_name <- file_names[i]
  is_editorial <- is_editorial_values[i]
  type <- type_values[i]
  df <- read_xlsx(file_name)
  df$is_editorial <- is_editorial
  df$type <- type
  df_list[[i]] <- df
}

df <- Reduce(function(x, y) merge(x, y, all = TRUE), df_list)


Data management

In [ ]:
# Scale values from 0 to 1
df$nb_clusters_norm <- rescale(nb_clusters, df, kind='0to1')
df$hhi_norm <- rescale(hhi, df, kind='0to1')
df$dist_norm <- rescale(mean_distance, df, kind='0to1')

# Print scaled values
df$nb_clusters_norm
df$hhi_norm
df$dist_norm


In [ ]:
# Scale covariates
df$followers_res <- rescale(playlist_followers, df, kind='z')
df$followers_norm <- rescale(followers_res, df, kind='0to1')
df$followers_norm

df$track_pop_norm <- rescale(track_pop, df, kind='0to1')
df$track_pop_norm

df$artist_pop_norm <- rescale(artist_pop, df, kind='0to1')
df$artist_pop_norm

df$nb_tracks_res <- rescale(nb_tracks, df, kind='z')
df$nb_tracks_norm <- rescale(nb_tracks_res, df, kind='0to1')
df$nb_tracks_norm

df$nb_artists_norm <- rescale(nb_artists, df, kind='0to1')
df$nb_artists_norm


Descriptive stats

In [ ]:
nrow(df)


In [ ]:
summary(df$nb_clusters_norm)
summary(df$hhi_norm)
summary(df$dist_norm)


In [ ]:
ggplot(df, aes(x = nb_clusters_norm)) + 
  geom_histogram(binwidth = 0.025, color = "black", fill = "#a7a7f9") +
  xlim(0, 1) +
  theme_bw() +
  theme(panel.grid.major.y = element_line(linetype = "dotted"),
        panel.grid.minor.y = element_blank(),
        panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank(),
        aspect.ratio = 0.8) +
  labs(x = 'Diversity Index', y = 'Frequency')


In [ ]:
ggplot(df, aes(x = hhi_norm)) + 
  geom_histogram(binwidth = 0.025, color = "black", fill = "#a7a7f9") +
  xlim(0, 1) +
  theme_bw() +
  theme(panel.grid.major.y = element_line(linetype = "dotted"),
        panel.grid.minor.y = element_blank(),
        panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank(),
        aspect.ratio = 0.8) +
  labs(x = 'Diversity Index', y = 'Frequency')

In [ ]:
ggplot(df, aes(x = dist_norm)) + 
  geom_histogram(binwidth = 0.025, color = "black", fill = "#a7a7f9") +
  xlim(0, 1) +
  theme_bw() +
  theme(panel.grid.major.y = element_line(linetype = "dotted"),
        panel.grid.minor.y = element_blank(),
        panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank(),
        aspect.ratio = 0.8) +
  labs(x = 'Diversity Index', y = 'Frequency')

In [ ]:
sapply(df[, c('nb_tracks', 'playlist_followers', 'track_pop', 'artist_pop')], function(x) c(summary(x), sd(x)))


In [ ]:
table(df$is_editorial)


In [ ]:
# Adjust options to display all columns of the data frame
options(width = 10000)

# Convert columns 15 and 16 to numeric
df_3[, 15:16] <- lapply(df_3[, 15:16], as.numeric)

# Create a new data frame to store scaled values
scaled_df <- data.frame(df_3)

# Rescale loudness, tempo, and duration to the range 0-1
scaled_df$loudness <- scale(scaled_df$loudness, center = min(scaled_df$loudness), scale = max(scaled_df$loudness) - min(scaled_df$loudness))
scaled_df$tempo <- scale(scaled_df$tempo, center = min(scaled_df$tempo), scale = max(scaled_df$tempo) - min(scaled_df$tempo))
scaled_df$duration_ms <- scale(scaled_df$duration_ms, center = min(scaled_df$duration_ms), scale = max(scaled_df$duration_ms) - min(scaled_df$duration_ms))

# Subset your scaled dataframe to select the tracks you want to compare
selected_tracks <- scaled_df[scaled_df$track_name %in% c("I Feel It Coming", "November Rain"), ]

# Print the values for the selected tracks
for (track_name in unique(selected_tracks$track_name)) {
  track_data <- selected_tracks[selected_tracks$track_name == track_name, 7:16]
  cat("Track:", track_name, "\n")
  print(track_data)
  cat("\n")
}


OLS models

Simple OLS

In [ ]:
lm_1 <- lm(nb_clusters_norm ~ followers_norm + nb_tracks_norm + track_pop_norm + artist_pop_norm + type, data = df)
summary(lm_1)

lm_2 <- lm(hhi_norm ~ followers_norm + nb_tracks_norm + track_pop_norm + artist_pop_norm + type, data = df)
summary(lm_2)

lm_3 <- lm(dist_norm ~ followers_norm + nb_tracks_norm + track_pop_norm + artist_pop_norm + type, data = df)
summary(lm_3)


In [ ]:
# Export regressions in LaTeX
stargazer(lm_1, lm_2, lm_3)


In [ ]:
# Export regressions in LaTeX
stargazer(lm_2)


Robustness checks

In [18]:
# create a list of file names and corresponding is_editorial values
file_names <- c("robust_afro.xlsx", "robust_alternatif.xlsx", "robust_ambiance.xlsx", "robust_ambient.xlsx", "robust_amour.xlsx",
                "robust_arabe.xlsx", "robust_bienetre.xlsx", "robust_blues.xlsx", "robust_classique.xlsx", "robust_country.xlsx",
                "robust_decennies.xlsx", "robust_detente.xlsx", "robust_dormir.xlsx", "robust_electro.xlsx")

df_list <- list()
for (i in seq_along(file_names)) {
  file_name <- file_names[i]
  df <- read_xlsx(file_name)
  df_list[[i]] <- df
}

df <- Reduce(function(x, y) merge(x, y, all = TRUE), df_list)

New names:
* `` -> `...1`
New names:
* `` -> `...1`
New names:
* `` -> `...1`
New names:
* `` -> `...1`
New names:
* `` -> `...1`
New names:
* `` -> `...1`
New names:
* `` -> `...1`
New names:
* `` -> `...1`
New names:
* `` -> `...1`
New names:
* `` -> `...1`
New names:
* `` -> `...1`
New names:
* `` -> `...1`
New names:
* `` -> `...1`
New names:
* `` -> `...1`


In [19]:
t.test(df$silhouette_score_5, df$silhouette_score_4)

t.test(df$silhouette_score_5_2, df$silhouette_score_4_2)



	Welch Two Sample t-test

data:  df$silhouette_score_5 and df$silhouette_score_4
t = -6.516, df = 918, p-value = 0.0000000001189
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -0.03292932 -0.01768493
sample estimates:
mean of x mean of y 
0.2627609 0.2880680 



	Welch Two Sample t-test

data:  df$silhouette_score_5_2 and df$silhouette_score_4_2
t = -13.103, df = 915.29, p-value < 0.00000000000000022
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -0.03897086 -0.02881754
sample estimates:
mean of x mean of y 
0.2392837 0.2731779 


In [ ]:
# Perform OLS
lm_3_1 <- lm(div_share_norm ~ followers_norm + nb_tracks_norm + track_pop_norm + artist_pop_norm + type, data = df)

# Display regression
summary(lm_3_1)

lm_3_2 <- lm(div_dist_norm ~ followers_norm + nb_tracks_norm + track_pop_norm + artist_pop_norm + type, data = df)

# Display regression
summary(lm_3_2)
